In [1]:
import pytagcloud
import collections
from konlpy.tag import Okt
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
from tqdm import tqdm
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, MLEProbDist
import warnings
warnings.simplefilter("ignore")

tagger = Okt()

def tokenize(data):
    tokens = ['/'.join(t) for t in tagger.pos(data)]
    return tokens

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# totalBigram은 있어야 함, finalWord를 갱신하고 싶으면 여기부터 돌리고, 둘 다 있으면 밑으로 내려가서 시작

In [2]:
with open('total_bigram.json', 'r') as jsonFile:
    totalBigram = json.load(jsonFile)
#미리 저장한 totalBigram, 불러오는데 시간 좀 걸림

totalTokens = [token[0] for token in totalBigram]
#bigram에서 token 불러오기

tokenCount = collections.Counter(totalTokens)
sortedToken = sorted(tokenCount.items(), key=lambda x: x[1], reverse=True)
sortedTokenDict = {}
for i in range(len(sortedToken)):
    sortedTokenDict[sortedToken[i][0]] = sortedToken[i][1]
#토큰의 빈도 수로 만든 dictionary

totalCfd = ConditionalFreqDist(totalBigram)
#조건부확률 뭐시기인데, 이거 통해서 관련 단어 뽑아냄

In [3]:
def korean_most_common(c, n, pos=None):
    if pos is None:
        return totalCfd[tokenize(c)[0]].most_common(n)
    else:
        return totalCfd["/".join([c, pos])].most_common(n)

In [4]:
word_list = ['형/Noun', '형/Suffix', '형님/Noun', '이형/Noun','누나/Noun','언니/Suffix', '언니/Noun',
             '대충/Noun', '브로/Noun', '브로앤팁스/Noun', '남자/Noun',
             '대충/Noun', '얼굴/Noun', '피부/Noun', '제품/Noun', '트러블/Noun']

#########################################중요 수정사항 많을 듯########################################

def wordRatio(word):
    kmc = korean_most_common(word, 1000)
    test = {} ##비율이 들어갈 딕셔너리
    for w in kmc:
        try:
            if int(sortedTokenDict[w[0]]) < 10:
                pass
            else:    
                test[w[0]] = int(w[1])/int(sortedTokenDict[w[0]])
        except:
            continue
    sort = sorted(test.items(), key=lambda x: x[1], reverse=True)[:30]
    new = []
    for t in sort:
        if float(t[1]) < 0.1:
            pass
        else:
            new.append(t)
    return new

In [5]:
final = []
for word in word_list:
    final.extend(wordRatio(word))
    
lastWord = []
for w in final:
    lastWord.append(w[0])    
lastWord.extend(word_list)

finalWord = list(set(lastWord))

with open('finalWord.json', 'w', encoding = 'utf-8') as jsonFile:
    json.dump(finalWord, jsonFile)

# 여기서부터 시작

In [6]:
df = pd.read_csv('53youtubers.csv', encoding = 'utf-8-sig')
youtuber = list(df.youtuber[i] for i in range(len(df)))

In [7]:
with open('finalWord.json', 'r') as jsonFile:
    finalWord = json.load(jsonFile)

In [8]:
def finalCount(youtuber):
    tokens = []
    df = pd.read_csv('{0}.csv'.format(youtuber), encoding = 'utf-8-sig', engine = 'python')
    for i in tqdm(range(len(df))):
        try:
            tokens.extend(tokenize(df.text[i]))
        except TypeError as e:
            print(e)
    finalToken = [w for w in tokens if w in finalWord]
    finalCount = sorted(collections.Counter(finalToken).items(), key=lambda x: x[1], reverse=True)
    return finalCount

In [9]:
finalCount('JM')

100%|███████████████████████████████████████████████████████████████████████████| 19008/19008 [01:13<00:00, 257.81it/s]


[('형/Noun', 2069),
 ('제품/Noun', 211),
 ('얼굴/Noun', 182),
 ('형님/Noun', 160),
 ('남자/Noun', 140),
 ('이형/Noun', 110),
 ('형/Suffix', 82),
 ('누나/Noun', 67),
 ('대충/Noun', 22),
 ('명/Suffix', 21),
 ('언니/Noun', 19),
 ('언니/Suffix', 12),
 ('불매운동/Noun', 12),
 ('피부/Noun', 10),
 ('주접/Noun', 10),
 ('힘내/Verb', 7),
 ('보야/Adjective', 4),
 ('왁싱/Noun', 4),
 ('참으세요/Verb', 2),
 ('망했어/Adjective', 2),
 ('동차/Noun', 2),
 ('만나야지/Verb', 1),
 ('소멸/Noun', 1),
 ('살찐거/Adjective', 1),
 ('노화/Noun', 1),
 ('브로/Noun', 1),
 ('살자/Verb', 1),
 ('살쪘다/Adjective', 1),
 ('가봐도/Verb', 1),
 ('수배/Noun', 1),
 ('먹는거만/Verb', 1),
 ('일것이다/Verb', 1),
 ('만나고싶다/Verb', 1),
 ('수발/Noun', 1),
 ('감추는/Verb', 1),
 ('밝히는/Verb', 1),
 ('왔구나/Verb', 1),
 ('증맬루/Noun', 1),
 ('주름살/Noun', 1),
 ('뵙고싶습니다/Verb', 1),
 ('과로사/Noun', 1),
 ('솔직히말해/Adjective', 1),
 ('죽지마/Verb', 1),
 ('추해/Adjective', 1)]